In [ ]:
!pip install sympy gmpy2 numpy


In [ ]:
import gmpy2
from gmpy2 import mpz, is_prime, gcd, isqrt, log, exp, sqrt
import numpy as np

def calcular_B(N):
    logN = log(N)
    loglogN = log(logN)
    B = exp(0.5 * sqrt(logN * loglogN))
    return int(B)

def quadratic_sieve(n, M=2000):
    n = mpz(n)  # Converter para mpz para lidar com números grandes

    # Passo 1: Calcular o limite superior B para os primos
    B = calcular_B(n)
    primes = [mpz(p) for p in range(2, B + 1) if is_prime(p)]

    # Informações solicitadas
    limite_superior = B
    num_primos = len(primes)

    # Passo 2: Escolher M valores de x e calcular f(x) = (x + sqrt(n))^2 - n
    sqrt_n = isqrt(n)
    x_values = [sqrt_n + i for i in range(M)]
    f_values = [(x**2 - n) for x in x_values]

    # Tamanho do vetor de índices j
    tamanho_vetor_j = len(f_values)

    # Passo 3: Fatorar f(x) na base escolhida
    factor_matrix = []
    for f in f_values:
        factorization = []
        for p in primes:
            exp = 0
            while f % p == 0:
                f //= p
                exp += 1
            factorization.append(exp % 2)  # modulo 2
        factor_matrix.append(factorization)
    factor_matrix = np.array(factor_matrix)

    # Passo 4: Resolver o sistema linear para encontrar uma combinação de fatores
    _, _, v = np.linalg.svd(factor_matrix)
    solution = v[-1]  # menor vetor singular

    # Evitar a solução trivial
    if not any(solution):
        solution = v[-2]  # usar outro vetor singular

    # Passo 5: Determinar os fatores de n
    factors = []
    for i, bit in enumerate(solution):
        if bit != 0:
            factors.append(x_values[i])
    if not factors:
        raise ValueError("Nenhuma solução não-trivial encontrada.")

    x_prod = np.prod(factors) % n

    # Usando a função isqrt para lidar com números grandes
    y_squared = abs(np.prod([pow(x, 2) - n for x in factors])) % n
    y_prod = isqrt(y_squared) % n

    factor1 = gcd(x_prod - y_prod, n)
    factor2 = gcd(x_prod + y_prod, n)

    # Saídas solicitadas
    print(f"Limite superior para os primos: {limite_superior}")
    print(f"Número de primos: {num_primos}")
    print(f"Tamanho do vetor de índices j: {tamanho_vetor_j}")
    print(f"x = {x_prod}, y = {y_prod}")
    print(f"mdc(x - y, N) = {factor1}")
    print(f"mdc(x + y, N) = {factor2}")

    return x_prod, y_prod, factor1, factor2

# Teste com o número a ser fatorado
n = '999999999000000001'  # Exemplo de número a ser fatorado
quadratic_sieve(n)


Limite superior para os primos: 499
Número de primos: 95
Tamanho do vetor de índices j: 2000
x = 629727864227498253, y = 941873193
mdc(x - y, N) = 1
mdc(x + y, N) = 1


(mpz(629727864227498253), mpz(941873193), mpz(1), mpz(1))